In [1]:
import pandas as pd
import math
import os

def process_p_values_sheet_corrected(input_file, sheet_name_to_process='P_Values'):
    """
    Processes the input Excel file:
    1. Reads the specified sheet (e.g., 'P_Values').
    2. Splits this sheet's data into new sheets (e.g., 'group_0', 'group_1')
       based on 400-row segments of the original 'sheet_name_to_process'.
       Each new 'group_X' sheet will be named based on its segment index.
    3. For each new 'group_X' sheet, it takes the 'p_value_0' column data
       from its corresponding 400-row segment of the original sheet,
       transposes every 4 valid (non-NaN) values into new columns
       ('p_value_0', 'p_value_1', 'p_value_2', 'p_value_3'),
       and writes this reshaped data to the 'group_X' sheet.
    Other original sheets from the input file are preserved. The 'sheet_name_to_process'
    itself is effectively replaced by the new 'group_X' sheets in the output.

    Args:
        input_file (str): The path to the input Excel file.
        sheet_name_to_process (str): The name of the sheet to process (default: 'P_Values').
    """
    try:
        all_original_sheets_data = pd.read_excel(input_file, sheet_name=None)
        print(f"Successfully loaded all sheets from '{input_file}'.")

        if sheet_name_to_process not in all_original_sheets_data:
            print(f"Error: Target sheet '{sheet_name_to_process}' not found in '{input_file}'.")
            return
        
        df_to_process = all_original_sheets_data.pop(sheet_name_to_process)
        print(f"Extracted '{sheet_name_to_process}' for processing. It will be replaced by 'group_X' sheets.")

    except FileNotFoundError:
        print(f"Error: The file '{input_file}' was not found.")
        return
    except Exception as e:
        print(f"An error occurred while reading the Excel file: {e}")
        return

    processed_group_sheets_data = {}

    if 'p_value_0' not in df_to_process.columns:
        print(f"Error: 'p_value_0' column not found in the sheet '{sheet_name_to_process}'. "
              "Cannot create 'group_X' sheets as specified.")
    else:
        total_original_rows = len(df_to_process)
        rows_per_group_source_chunk = 400 
        
        if total_original_rows == 0:
            print(f"Warning: The sheet '{sheet_name_to_process}' is empty. "
                  "No 'group_X' sheets will be generated from it.")
        else:
            num_group_sheets = math.ceil(total_original_rows / rows_per_group_source_chunk)
            print(f"Original sheet '{sheet_name_to_process}' has {total_original_rows} rows. "
                  f"Planning to create {num_group_sheets} 'group_X' sheet(s).")

            for i in range(num_group_sheets):
                group_sheet_name_output = f'group_{i}'
                start_row_index = i * rows_per_group_source_chunk
                end_row_index = min((i + 1) * rows_per_group_source_chunk, total_original_rows)
                
                current_chunk_df = df_to_process.iloc[start_row_index:end_row_index]
                print(f"  Processing chunk for '{group_sheet_name_output}': "
                      f"original rows {start_row_index} to {end_row_index - 1}")

                p_value_0_segment = current_chunk_df['p_value_0'].dropna()

                if p_value_0_segment.empty:
                    print(f"    'p_value_0' data in this chunk for '{group_sheet_name_output}' "
                          "is empty or all NaNs. Sheet will have headers only.")
                    reshaped_group_data = pd.DataFrame(columns=[f'p_value_{j}' for j in range(4)])
                else:
                    num_p_values_in_segment = len(p_value_0_segment)
                    
                    if num_p_values_in_segment % 4 != 0:
                        padding_needed = 4 - (num_p_values_in_segment % 4)
                        p_value_0_segment = pd.concat(
                            [p_value_0_segment, pd.Series([pd.NA] * padding_needed)], 
                            ignore_index=True 
                        )
                    
                    num_reshaped_rows = len(p_value_0_segment) // 4
                    
                    if num_reshaped_rows > 0:
                        reshaped_values_array = p_value_0_segment.values.reshape(num_reshaped_rows, 4)
                        reshaped_group_data = pd.DataFrame(reshaped_values_array, 
                                                           columns=[f'p_value_{j}' for j in range(4)])
                    else: 
                        reshaped_group_data = pd.DataFrame(columns=[f'p_value_{j}' for j in range(4)])
                    
                    print(f"    Reshaped 'p_value_0' data for '{group_sheet_name_output}' "
                          f"into {len(reshaped_group_data)} rows and 4 columns.")
                
                processed_group_sheets_data[group_sheet_name_output] = reshaped_group_data
    
    # --- Writing to Excel ---
    base, ext = os.path.splitext(input_file)
    output_file = f"{base}{ext}" 
    
    # !!! IMPORTANT: If you want to OVERWRITE your original file, uncomment the next line
    # AND comment out the two lines above this comment.
    # output_file = input_file 

    try:
        with pd.ExcelWriter(output_file, engine='openpyxl', mode='w') as writer:
            if all_original_sheets_data:
                for original_sheet_name, original_df in all_original_sheets_data.items():
                    original_df.to_excel(writer, sheet_name=original_sheet_name, index=False)
                    print(f"Preserved original sheet: '{original_sheet_name}'")
            
            if processed_group_sheets_data:
                for grp_name, grp_df in processed_group_sheets_data.items():
                    grp_df.to_excel(writer, sheet_name=grp_name, index=False)
                    print(f"Saved processed data to new sheet: '{grp_name}'")
            elif 'p_value_0' not in df_to_process.columns and sheet_name_to_process in all_original_sheets_data:
                 print(f"No 'group_X' sheets created as 'p_value_0' column was missing in '{sheet_name_to_process}'.")
            elif len(df_to_process) == 0 and sheet_name_to_process in all_original_sheets_data: # df_to_process could be empty
                 print(f"No 'group_X' sheets created as '{sheet_name_to_process}' was empty.")
            elif 'p_value_0' in df_to_process.columns and len(df_to_process) > 0:
                print(f"No data was available in 'p_value_0' of '{sheet_name_to_process}' to populate 'group_X' sheets "
                      "(e.g., all values were NaN). 'group_X' sheets might be empty if listed above.")

        if output_file == input_file:
            print(f"Processing complete. Original file '{output_file}' has been overwritten with changes.")
        else:
            print(f"Processing complete. New file created at '{output_file}'.")

    except Exception as e:
        print(f"An error occurred while writing to the Excel file '{output_file}': {e}")
        if output_file == input_file and os.path.exists(input_file):
            print(f"WARNING: The original file '{input_file}' might be corrupted or "
                  "partially written due to the error.")


In [5]:


input_excel_files = ["did_dr_GATE_and_PValues_linearity_degree=1.xlsx","did_dr_GATE_and_PValues_linearity_degree=2.xlsx","did_dr_GATE_and_PValues_linearity_degree=3.xlsx","did_dr_GATE_PS_and_PValues_linearity_degree=1.xlsx","did_dr_GATE_PS_and_PValues_linearity_degree=2.xlsx","did_dr_GATE_PS_and_PValues_linearity_degree=3.xlsx","did_dr_CATE_GATE_PS_and_PValues_linearity_degree=1.xlsx","did_dr_CATE_GATE_PS_and_PValues_linearity_degree=2.xlsx","did_dr_CATE_GATE_PS_and_PValues_linearity_degree=3.xlsx","DoubleML_did_GATE_and_PValues_linearity_degree=1.xlsx","DoubleML_did_GATE_and_PValues_linearity_degree=2.xlsx","DoubleML_did_GATE_and_PValues_linearity_degree=3.xlsx","DoubleML_did_GATE_PS_and_PValues_linearity_degree=1.xlsx","DoubleML_did_GATE_PS_and_PValues_linearity_degree=2.xlsx","DoubleML_did_GATE_PS_and_PValues_linearity_degree=3.xlsx","DoubleML_did_CATE_GATE_PS_and_PValues_linearity_degree=1.xlsx","DoubleML_did_CATE_GATE_PS_and_PValues_linearity_degree=2.xlsx","DoubleML_did_CATE_GATE_PS_and_PValues_linearity_degree=3.xlsx","did2s_GATE_and_PValues_linearity_degree=1.xlsx","did2s_GATE_and_PValues_linearity_degree=2.xlsx","did2s_GATE_and_PValues_linearity_degree=3.xlsx","synthdid_GATE_and_PValues_linearity_degree=1.xlsx","synthdid_GATE_and_PValues_linearity_degree=2.xlsx","synthdid_GATE_and_PValues_linearity_degree=3.xlsx","synthdid_GATE_PS_and_PValues_linearity_degree=1.xlsx","synthdid_GATE_PS_and_PValues_linearity_degree=2.xlsx","synthdid_GATE_PS_and_PValues_linearity_degree=3.xlsx","synthdid_CATE_GATE_PS_and_PValues_linearity_degree=1.xlsx","synthdid_CATE_GATE_PS_and_PValues_linearity_degree=2.xlsx","synthdid_CATE_GATE_PS_and_PValues_linearity_degree=3.xlsx"]
sheet_to_process = "P_Values"  # Change this if your sheet has a different name

# Call the function
for i in range(len(input_excel_files)):
    process_p_values_sheet_corrected(input_excel_files[i], sheet_name_to_process=sheet_to_process)

print("\nScript finished.")

Successfully loaded all sheets from 'did_dr_GATE_and_PValues_linearity_degree=1.xlsx'.
Extracted 'P_Values' for processing. It will be replaced by 'group_X' sheets.
Original sheet 'P_Values' has 1200 rows. Planning to create 3 'group_X' sheet(s).
  Processing chunk for 'group_0': original rows 0 to 399
    Reshaped 'p_value_0' data for 'group_0' into 100 rows and 4 columns.
  Processing chunk for 'group_1': original rows 400 to 799
    Reshaped 'p_value_0' data for 'group_1' into 100 rows and 4 columns.
  Processing chunk for 'group_2': original rows 800 to 1199
    Reshaped 'p_value_0' data for 'group_2' into 100 rows and 4 columns.
Preserved original sheet: 'Metrics'
Saved processed data to new sheet: 'group_0'
Saved processed data to new sheet: 'group_1'
Saved processed data to new sheet: 'group_2'
Processing complete. Original file 'did_dr_GATE_and_PValues_linearity_degree=1.xlsx' has been overwritten with changes.
Successfully loaded all sheets from 'did_dr_GATE_and_PValues_linear